In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak
from sklearn.model_selection import train_test_split


In [127]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,TEST_6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,TEST_6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,TEST_6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [117]:
# test.COMPONENT_ARBITRARY.value_counts()
# test.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [162]:
# test = pd.read_csv('./test.csv')
# test = test[test['COMPONENT_ARBITRARY'] == 'COMPONENT3']
# tp = test.loc[:,['ID']]
# test = test.drop(['ID','COMPONENT_ARBITRARY'],axis=1)
# test.to_csv('./test_target3.csv')

In [17]:
train.COMPONENT_ARBITRARY.value_counts()

COMPONENT3    7050
COMPONENT1    3890
COMPONENT2    2316
COMPONENT4     839
Name: COMPONENT_ARBITRARY, dtype: int64

In [142]:
def make_train_data(x):
    train = pd.read_csv('./train.csv')
    comp4 = train[train['COMPONENT_ARBITRARY'] == f'COMPONENT{x}']
    tp = comp4.loc[:,['ID', 'COMPONENT_ARBITRARY']]
    target = ['ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
           'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
           'V40', 'ZN','Y_LABEL']
    comp4 = comp4.loc[:,target]
    comp4.info()

    train4, test4 = train_test_split(comp4, test_size=0.15, random_state=10)
    train4.to_csv(f'./comp{x}_train.csv')
    test4.to_csv(f'./comp{x}_test.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7050 entries, 0 to 14092
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ANONYMOUS_1  7050 non-null   int64  
 1   YEAR         7050 non-null   int64  
 2   ANONYMOUS_2  7050 non-null   int64  
 3   AG           7050 non-null   int64  
 4   CO           7050 non-null   int64  
 5   CR           7050 non-null   int64  
 6   CU           7050 non-null   int64  
 7   FE           7050 non-null   int64  
 8   H2O          7050 non-null   float64
 9   MN           7050 non-null   int64  
 10  MO           7050 non-null   int64  
 11  NI           7050 non-null   int64  
 12  PQINDEX      7050 non-null   int64  
 13  TI           7050 non-null   int64  
 14  V            7050 non-null   int64  
 15  V40          7050 non-null   float64
 16  ZN           7050 non-null   int64  
 17  Y_LABEL      7050 non-null   int64  
dtypes: float64(2), int64(16)
memory usage: 1.0 MB


In [181]:
pd.read_csv('./comp3_train.csv').Y_LABEL.value_counts()

0    5135
1     505
Name: Y_LABEL, dtype: int64

In [182]:
pd.read_csv('./comp3_test.csv').Y_LABEL.value_counts()

0    1280
1     130
Name: Y_LABEL, dtype: int64

In [287]:
def model_pred(x):
        # Initialize the structured data classifier.
    clf = ak.StructuredDataClassifier(
        overwrite=True, max_trials=4
    )  # It tries 3 different models.
    # Feed the structured data classifier with training data.
    clf.fit(
        # The path to the train.csv file.
        f'./comp{x}_train.csv',
        # The name of the label column.
        "Y_LABEL",
        epochs=20,
    )
    # Predict with the best model.
    predicted_y = clf.predict(f'./comp{x}_test.csv')
    # Evaluate the best model with testing data.
    # print(clf.evaluate(f'./comp{x}_test.csv', "Y_LABEL"))
    model = clf.export_model()
    pred = model.predict(pd.read_csv(f'./test_target{x}.csv'))

    return pred

In [288]:
def make_ans(x,pred):
    test = pd.read_csv('./test.csv')
    test = test[test['COMPONENT_ARBITRARY'] == f'COMPONENT{x}']
    tp = test.loc[:,['ID']]
    
    aa = []
    for i in pred:
        if i[0] > 0.3:
            aa.append(1)
        else:
            aa.append(0)
    
    tp['Y_LABEL'] = aa
    tp.to_csv(f'./comp{x}_ans.csv')

In [289]:
for i in range(1,5):
    make_ans(i,model_pred(i))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.9212598204612732

Best val_accuracy So Far: 0.9370078444480896
Total elapsed time: 00h 00m 17s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
21/21 [==============================] - 1s 2ms/step - loss: 0.5041 - accuracy: 0.8331
Epoch 2/20
21/21 [==============================] - 0s 1ms/step - loss: 0.3437 - accuracy: 0.9419
Epoch 3/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2866 - accuracy: 0.9419
Epoch 4/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2588 - accuracy: 0.9419
Epoch 5/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.9419
Epoch 6/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2306 - accuracy: 0.9419
Epoch 7/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2214 - accuracy: 0.9419
Epoch 8/20
21/21 [==============================] - 0s 1ms/step - loss: 0.2140 - accuracy: 0.9419
Epoch 9/20
21/21 [==================

In [290]:
#def make_merge():
cp1 = pd.read_csv('./comp1_ans.csv')
cp2 = pd.read_csv('./comp2_ans.csv')
cp3 = pd.read_csv('./comp3_ans.csv')
cp4 = pd.read_csv('./comp4_ans.csv')
print(pd.concat([cp1,cp2,cp3,cp4],axis=0).sort_values(by='ID').set_index('ID').drop('Unnamed: 0',axis=1).value_counts('Y_LABEL'))
pd.concat([cp1,cp2,cp3,cp4],axis=0).sort_values(by='ID').set_index('ID').drop('Unnamed: 0',axis=1).to_csv('./comp_ans.csv')

Y_LABEL
0    5974
1      67
dtype: int64


In [169]:
pd.read_csv('./sample_submission.csv')

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [259]:
train.Y_LABEL.value_counts()

0    12892
1     1203
Name: Y_LABEL, dtype: int64

In [299]:
12892/len(train)

0.9146505853139412

In [236]:
train[train['Y_LABEL'] == 1].loc[:,target].CO.value_counts()

0    1164
1      34
2       5
Name: CO, dtype: int64

In [309]:
train = pd.read_csv('./train.csv')
target = ['ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
           'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
           'V40', 'ZN','Y_LABEL']
comp4 = train.loc[:,target]
train4, test4 = train_test_split(comp4, test_size=0.15, random_state=21)
train4.to_csv(f'./comp_train.csv')
test4.to_csv(f'./comp_test.csv')

target = ['ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
           'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
           'V40', 'ZN']
test = pd.read_csv('./test.csv')
tp = test.loc[:,['ID']]
test = test.loc[:,target]
test.to_csv('./test_target.csv')

    # Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    # column_types={"V": "categorical"},
    max_trials=10,  # It tries 10 different models.
    overwrite=True,
    # Include it as one of the metrics.
)
  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    f'./comp_train.csv',
    # The name of the label column.
    "Y_LABEL",
    epochs=500,
)
# Predict with the best model.
predicted_y = clf.predict(f'./comp_test.csv')
# Evaluate the best model with testing data.
# print(clf.evaluate(f'./comp{x}_test.csv', "Y_LABEL"))
model = clf.export_model()
pred = model.predict(pd.read_csv('./test_target.csv'))

Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.9159663915634155

Best val_accuracy So Far: 0.9163865447044373
Total elapsed time: 00h 05m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/500
375/375 [==============================] - 1s 1ms/step - loss: 0.3117 - accuracy: 0.9113
Epoch 2/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2894 - accuracy: 0.9144
Epoch 3/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2860 - accuracy: 0.9149
Epoch 4/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2837 - accuracy: 0.9150
Epoch 5/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2817 - accuracy: 0.9152
Epoch 6/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2801 - accuracy: 0.9154
Epoch 7/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2787 - accuracy: 0.9155
Epoch 8/500
375/375 [==============================] - 1s 1ms/step - loss: 0.2773 - accuracy: 0.9159
Epoch 9/500

375/375 [==============================] - 0s 1ms/step - loss: 0.2197 - accuracy: 0.9279
Epoch 80/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2180 - accuracy: 0.9280
Epoch 81/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2171 - accuracy: 0.9285
Epoch 82/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2167 - accuracy: 0.9286
Epoch 83/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2159 - accuracy: 0.9287
Epoch 84/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2154 - accuracy: 0.9291
Epoch 85/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2151 - accuracy: 0.9294
Epoch 86/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2143 - accuracy: 0.9290
Epoch 87/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2135 - accuracy: 0.9296
Epoch 88/500
375/375 [==============================] - 0s 1ms/step - loss: 0.2134 - accuracy: 

375/375 [==============================] - 1s 1ms/step - loss: 0.1700 - accuracy: 0.9427
Epoch 159/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1699 - accuracy: 0.9429
Epoch 160/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1696 - accuracy: 0.9433
Epoch 161/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1688 - accuracy: 0.9432
Epoch 162/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1685 - accuracy: 0.9434
Epoch 163/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1686 - accuracy: 0.9437
Epoch 164/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1682 - accuracy: 0.9434
Epoch 165/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1671 - accuracy: 0.9438
Epoch 166/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1665 - accuracy: 0.9444
Epoch 167/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1663 - a

375/375 [==============================] - 1s 1ms/step - loss: 0.1477 - accuracy: 0.9483
Epoch 238/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1427 - accuracy: 0.9509
Epoch 239/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1390 - accuracy: 0.9526
Epoch 240/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1380 - accuracy: 0.9532
Epoch 241/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1345 - accuracy: 0.9541
Epoch 242/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1347 - accuracy: 0.9538
Epoch 243/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1337 - accuracy: 0.9544
Epoch 244/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1331 - accuracy: 0.9538
Epoch 245/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1330 - accuracy: 0.9539
Epoch 246/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1332 - a

375/375 [==============================] - 1s 1ms/step - loss: 0.1134 - accuracy: 0.9629
Epoch 317/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1129 - accuracy: 0.9616
Epoch 318/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1129 - accuracy: 0.9625
Epoch 319/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1124 - accuracy: 0.9617
Epoch 320/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1126 - accuracy: 0.9623
Epoch 321/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1119 - accuracy: 0.9629
Epoch 322/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1118 - accuracy: 0.9628
Epoch 323/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1126 - accuracy: 0.9621
Epoch 324/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1284 - accuracy: 0.9565
Epoch 325/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1475 - a

375/375 [==============================] - 1s 1ms/step - loss: 0.1044 - accuracy: 0.9659
Epoch 396/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1261 - accuracy: 0.9570
Epoch 397/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1092 - accuracy: 0.9631
Epoch 398/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0957 - accuracy: 0.9684
Epoch 399/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0945 - accuracy: 0.9690
Epoch 400/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0944 - accuracy: 0.9691
Epoch 401/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0946 - accuracy: 0.9694
Epoch 402/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0964 - accuracy: 0.9690
Epoch 403/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0947 - accuracy: 0.9685
Epoch 404/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0940 - a

375/375 [==============================] - 1s 1ms/step - loss: 0.0917 - accuracy: 0.9697
Epoch 475/500
375/375 [==============================] - 1s 1ms/step - loss: 0.1100 - accuracy: 0.9649
Epoch 476/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0933 - accuracy: 0.9699
Epoch 477/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0859 - accuracy: 0.9711
Epoch 478/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0852 - accuracy: 0.9708
Epoch 479/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0854 - accuracy: 0.9721
Epoch 480/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0809 - accuracy: 0.9735
Epoch 481/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0812 - accuracy: 0.9735
Epoch 482/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0822 - accuracy: 0.9730
Epoch 483/500
375/375 [==============================] - 1s 1ms/step - loss: 0.0821 - a

In [311]:
aa = []
for i in pred:
    if i[0] > 0.5:
        aa.append(1)
    else:
        aa.append(0)

tp['Y_LABEL'] = aa
# tp = tp.set_index('ID')
tp.to_csv('./comp_ans.csv')
pd.read_csv('./comp_ans.csv').Y_LABEL.value_counts()

0    5476
1     565
Name: Y_LABEL, dtype: int64

In [301]:
12892/len(train) # 0 rate

0.9146505853139412

In [304]:
6041-6041*12892/len(train)

515.5958141184819